In [2]:
from detection import detect_grid_and_cells

In [3]:
MODEL_PATH = r"C:\Users\77019\Desktop\kbtu\5 sem\tic_tac_toe_cv\runs\detect\train3\weights\best.pt"

In [4]:
import cv2
import numpy as np
from ultralytics import YOLO
from detection import *  # Import helper functions from your file

# Load YOLO model
def load_yolo_model(model_path):
    return YOLO(model_path)

def get_original_image_cells(image_path, resize_dim=(600, 600), show_image=False):
    image = cv2.imread(image_path)

    # Detect the grid and get the perspective transformation matrix
    cells = detect_grid_and_cells(image_path, resize_dim=resize_dim, show_image=show_image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    contours, _ = cv2.findContours(cv2.adaptiveThreshold(gray, 255, 1, 1, 11, 2), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    max_area, best_cnt = 0, None
    for c in contours:
        area = cv2.contourArea(c)
        if area > 10000 and area > max_area:
            max_area = area
            best_cnt = c

    # Find four corners of the grid for transformation
    epsilon = 0.02 * cv2.arcLength(best_cnt, True)
    approx = cv2.approxPolyDP(best_cnt, epsilon, True)
    pts = np.array([point[0] for point in approx], dtype="float32")
    rect = np.zeros((4, 2), dtype="float32")
    s, diff = pts.sum(axis=1), np.diff(pts, axis=1)
    rect[0], rect[2] = pts[np.argmin(s)], pts[np.argmax(s)]
    rect[1], rect[3] = pts[np.argmin(diff)], pts[np.argmax(diff)]
    
    # Compute perspective transform and inverse transform matrices
    dst_pts = np.array([[0, 0], [resize_dim[0], 0], [resize_dim[0], resize_dim[1]], [0, resize_dim[1]]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst_pts)
    M_inv = cv2.getPerspectiveTransform(dst_pts, rect)

    # Apply perspective transform to get cells in transformed space
    warped_image = apply_perspective_transform(image, rect, resize_dim[0], resize_dim[1])
    transformed_cells = divide_into_cells((0, 0, resize_dim[0], resize_dim[1]), warped_image)

    # Transform cells back to original coordinates using M_inv
    original_cells = []
    for cell in transformed_cells:
        top_left = np.dot(M_inv, np.array([cell['x0'], cell['y0'], 1]))
        bottom_right = np.dot(M_inv, np.array([cell['x1'], cell['y1'], 1]))
        original_cells.append({
            'x0': int(top_left[0] / top_left[2]),
            'y0': int(top_left[1] / top_left[2]),
            'x1': int(bottom_right[0] / bottom_right[2]),
            'y1': int(bottom_right[1] / bottom_right[2])
        })

    return original_cells

# Function to detect X/O and identify corresponding cell
def detect_xo_and_identify_cells(frame, model, cells):
    results = model.predict(source=frame, conf=0.5, show=False, verbose=False)  # Adjust `conf` if necessary
    detected_objects = []

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
            label = result.names[int(box.cls)]
            confidence = float(box.conf)

            # Determine the cell that contains this object
            cell_index = find_cell_for_object((center_x, center_y), cells)
            
            if cell_index is not None:
                detected_objects.append({
                    'label': label,
                    'confidence': confidence,
                    'cell_index': cell_index,
                    'bbox': (x1, y1, x2, y2)
                })
    return detected_objects

# Helper function to find cell containing the center of the bounding box
def find_cell_for_object(center, cells):
    for index, cell in enumerate(cells):
        if cell['x0'] <= center[0] <= cell['x1'] and cell['y0'] <= center[1] <= cell['y1']:
            return index  # Return the cell index if the center is within cell bounds
    return None

# Function to annotate and display the results on the image
def annotate_and_display(image, detected_objects, original_cells, scale_factor=0.5, show_image=True):
    # Draw cell borders on the original image
    for cell in original_cells:
        x0, y0, x1, y1 = cell['x0'], cell['y0'], cell['x1'], cell['y1']
        cv2.rectangle(image, (x0, y0), (x1, y1), (255, 255, 255), 2)  # White border for cells

    # Annotate detected objects
    for obj in detected_objects:
        x1, y1, x2, y2 = obj['bbox']
        label = obj['label']
        cell_index = obj['cell_index']
        confidence = obj['confidence']
        
        # Draw bounding box and label on the image
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, f"{label} ({cell_index})", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        # print(f"Detected {label} in cell {cell_index} with confidence {confidence:.2f}")

    # Resize the image for display
    resized_image = cv2.resize(image, (int(image.shape[1] * scale_factor), int(image.shape[0] * scale_factor)))

    # Display the image with cells and detections if show_image is True
    if show_image:
        cv2.imshow("Tic Tac Toe Detection with Cells", resized_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [16]:
# Modify get_original_image_cells to accept an image array instead of path
def get_original_image_cells(image, resize_dim=(600, 600), show_image=False):
    """
    Detect grid and cells in an image (either as a path or image array) and return cell coordinates.
    """
    # Detect the grid and get the perspective transformation matrix
    cells = detect_grid_and_cells(image_path=None, image=image, resize_dim=resize_dim, show_image=show_image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    contours, _ = cv2.findContours(cv2.adaptiveThreshold(gray, 255, 1, 1, 11, 2), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    max_area, best_cnt = 0, None
    for c in contours:
        area = cv2.contourArea(c)
        if area > 10000 and area > max_area:
            max_area = area
            best_cnt = c

    # Find four corners of the grid for transformation
    try: 
        epsilon = 0.02 * cv2.arcLength(best_cnt, True)
        approx = cv2.approxPolyDP(best_cnt, epsilon, True)
        pts = np.array([point[0] for point in approx], dtype="float32")
        rect = np.zeros((4, 2), dtype="float32")
        s, diff = pts.sum(axis=1), np.diff(pts, axis=1)
        rect[0], rect[2] = pts[np.argmin(s)], pts[np.argmax(s)]
        rect[1], rect[3] = pts[np.argmin(diff)], pts[np.argmax(diff)]
        
        # Compute perspective transform and inverse transform matrices
        dst_pts = np.array([[0, 0], [resize_dim[0], 0], [resize_dim[0], resize_dim[1]], [0, resize_dim[1]]], dtype="float32")
        M = cv2.getPerspectiveTransform(rect, dst_pts)
        M_inv = cv2.getPerspectiveTransform(dst_pts, rect)

        # Apply perspective transform to get cells in transformed space
        warped_image = apply_perspective_transform(image, rect, resize_dim[0], resize_dim[1])
        transformed_cells = divide_into_cells((0, 0, resize_dim[0], resize_dim[1]), warped_image)

        # Transform cells back to original coordinates using M_inv
        original_cells = []
        for cell in transformed_cells:
            top_left = np.dot(M_inv, np.array([cell['x0'], cell['y0'], 1]))
            bottom_right = np.dot(M_inv, np.array([cell['x1'], cell['y1'], 1]))
            original_cells.append({
                'x0': int(top_left[0] / top_left[2]),
                'y0': int(top_left[1] / top_left[2]),
                'x1': int(bottom_right[0] / bottom_right[2]),
                'y1': int(bottom_right[1] / bottom_right[2])
            })

        return original_cells
    
    except:
        raise ValueError("grid not found")

In [15]:
import cv2
import numpy as np
from ultralytics import YOLO
from detection import *  # Import helper functions from your file
from collections import defaultdict
import time

# Load YOLO model
def load_yolo_model(model_path):
    return YOLO(model_path)

# Track the position stability for detected objects
def track_positions(detected_objects, frame_time, stable_positions, duration=2):
    confirmed_moves = []
    for obj in detected_objects:
        cell_index = obj['cell_index']
        label = obj['label']
        
        if (label, cell_index) in stable_positions:
            last_seen_time = stable_positions[(label, cell_index)]
            if frame_time - last_seen_time >= duration:
                confirmed_moves.append(obj)
        else:
            stable_positions[(label, cell_index)] = frame_time
    
    # Remove entries not seen in this frame
    stable_positions = {key: stable_positions[key] for key in stable_positions if key in {(obj['label'], obj['cell_index']) for obj in detected_objects}}
    return confirmed_moves, stable_positions

# Process live video feed and detect objects
def process_live_video(model_path, show_image=False):
    model = load_yolo_model(model_path)

    # Open live video feed (0 is usually the default webcam)
    video = cv2.VideoCapture("../videos/test_video.mp4")
    
    # Get grid cells from the first frame for reference
    ret, first_frame = video.read()
    if not ret:
        print("Error reading video.")
        return
    original_cells = get_original_image_cells(first_frame, show_image=show_image)

    # Initialize stable positions with the first frame detections
    initial_detections = detect_xo_and_identify_cells(first_frame, model, original_cells)
    frame_time = time.time()
    current_positions = {(obj['label'], obj['cell_index']): frame_time for obj in initial_detections}
    board = initial_state()

    # Loop over frames
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break
        frame_time = time.time()  # Current frame time
        
        detected_objects = detect_xo_and_identify_cells(frame, model, original_cells)
        confirmed_moves, current_positions = track_positions(
            detected_objects, 
            frame_time, 
            stable_positions=current_positions, 
            duration=2
        )
        
        if confirmed_moves:
            board = update_board(board, confirmed_moves)
            # Display the updated board and moves if needed
            print("Board updated:", board)
            
            # TODO Here we call Gholibs function to get next move. This will take some time. 
            # send_move_to_robot(next_move) пока не надо
            # wait for confirmation from robot пока не надо
            # TODO update board according to the move
            # TODO update current_positions according to the move
            # frame_time = time.time()
            # current_positions[next_move['label'], next_move['cell_index']] = frame_time

        if show_image:
            cv2.imshow('Real-Time Tic Tac Toe', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break  # Press 'q' to exit the loop

    video.release()
    cv2.destroyAllWindows()

X = "X"
O = "O"
EMPTY = None

def initial_state():
    """ Returns starting state of the board. """
    return [[EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY]
            ]

def update_board(board, confirmed_moves):
    for move in confirmed_moves:
        row, col = move['cell_index'] // 3, move['cell_index'] % 3
        board[row][col] = X if move['label'] == "x" else O
    return board 

# Example usage
process_live_video(MODEL_PATH, show_image=False)


Board updated: [['O', 'X', 'X'], ['O', 'O', 'X'], ['O', 'O', 'X']]
Board updated: [['O', 'X', 'X'], ['O', 'O', 'X'], ['O', 'O', 'X']]
Board updated: [['O', 'X', 'X'], ['O', 'O', 'X'], ['O', 'O', 'X']]
Board updated: [['O', 'X', 'X'], ['O', 'O', 'X'], ['O', 'O', 'X']]
Board updated: [['O', 'X', 'X'], ['O', 'O', 'X'], ['O', 'O', 'X']]
Board updated: [['O', 'X', 'X'], ['O', 'O', 'X'], ['O', 'O', 'X']]
Board updated: [['O', 'X', 'X'], ['O', 'O', 'X'], ['O', 'O', 'X']]
Board updated: [['O', 'X', 'X'], ['O', 'O', 'X'], ['O', 'O', 'X']]
Board updated: [['O', 'X', 'X'], ['O', 'O', 'X'], ['O', 'O', 'X']]
Board updated: [['O', 'X', 'X'], ['O', 'O', 'X'], ['O', 'O', 'X']]
Board updated: [['O', 'X', 'X'], ['O', 'O', 'X'], ['O', 'O', 'X']]
Board updated: [['O', 'X', 'X'], ['O', 'O', 'X'], ['O', 'O', 'X']]
Board updated: [['O', 'X', 'X'], ['O', 'O', 'X'], ['O', 'O', 'X']]
Board updated: [['O', 'X', 'X'], ['O', 'O', 'X'], ['O', 'O', 'X']]
Board updated: [['O', 'X', 'X'], ['O', 'O', 'X'], ['O', 'O', '